# Task 5: 

In [1]:
import pandas as pd 
import numpy as np
import joblib
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.svm import OneClassSVM
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
KERN = 'RBF'
NU, GAMMA = 0.01, 0.1

Load Data

In [3]:
df_raw = pd.read_csv('task5/test.csv')
df = df_raw.copy()
df.drop(['id', 'player_id'], axis=1, inplace=True)

Missing Value Imputation IterativeImputer

In [4]:
df_mice = df.copy(deep=True)

mice_imputer = IterativeImputer()
df_mice_array = mice_imputer.fit_transform(df_mice)  # ได้เป็น numpy array

df_mice = pd.DataFrame(df_mice_array, columns=df.columns)

d:\python\Lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Scaling data

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_mice)

Model 1: One-Class SVM

In [ ]:
ocsvm_clf = OneClassSVM(
    nu=NU,
    kernel=KERN.lower(),
    gamma=GAMMA
)

ocsvm_clf.fit(X_scaled)

# y_pred: 1 = inlier, -1 = outlier
y_ocsvm_pred = ocsvm_clf.predict(X_scaled)
ocsvm_scores = ocsvm_clf.decision_function(X_scaled)

pd.DataFrame({
    "ocsvm_pred": y_ocsvm_pred,
    "ocsvm_score": ocsvm_scores
}).to_csv("task5/predicted.csv", index=False)

joblib.dump(ocsvm_clf, 'task5/model.pkl')

['task5/model.pkl']

Model 2: Isolation Forest

In [7]:
iso_clf = IsolationForest(
    n_estimators=200,
    contamination="auto",
    random_state=42
)

iso_clf.fit(X_scaled)

iso_pred = iso_clf.predict(X_scaled)            # 1 = inlier, -1 = outlier
iso_scores = iso_clf.decision_function(X_scaled)  # ยิ่งมากยิ่งปกติ

In [8]:
joblib.dump(ocsvm_clf, 'task5/model.pkl')

['task5/model.pkl']

In [18]:
ocsvm_scores

array([ 0.000733  , -0.00023822, -0.00021748, ...,  0.00073296,
       -0.0002358 , -0.00022958], shape=(25889,))

In [19]:
save_y_ocsvm = pd.DataFrame(y_ocsvm_pred)

save_y_ocsvm.to_csv("task5/predicted_ocsvm.csv")

In [20]:
save_y_iso_pred = pd.DataFrame(iso_pred)

save_y_iso_pred.to_csv("task5/predicted_iso.csv")